In [2]:
from google.colab import drive

drive.mount("/content/C")

Mounted at /content/C


In [3]:
%%capture
!unzip /content/C/MyDrive/CS419.N11/Cranfield.zip -d Cranfield
!unzip /content/C/MyDrive/CS419.N11/TEST.zip -d Cranfield
!pip install Whoosh
!pip install pytrec_eval
!mkdir ind


**I) XỬ LÝ CÁC CÂU TRUY VẤN**

***1) LẬP CHỈ MỤC***
- Chọn term là dạng thân từ.
- Mô hình xác suất với trọng số term TF.IDF

In [4]:
import os
import nltk
from nltk import sent_tokenize
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

from whoosh.index import create_in
from whoosh.fields import *
from whoosh.analysis import StandardAnalyzer
from whoosh import qparser
from whoosh import scoring
import whoosh.index as index

import pytrec_eval
import math


nltk.download('punkt_tab')
nltk.download('stopwords')
stoplist = stopwords.words("english")
stoplist.append('oh')
puncts = ['.', ',', ':', '`', '"', "'", '!', '?', "``", "''"]
ps = PorterStemmer()

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [5]:
def preprocess(tok, stemmer=ps, punctlist=puncts, stopwords=stoplist):
  tok = tok.lower()
  if tok.isdigit():
    return None
  if tok.isnumeric():
    return None
  if tok in punctlist:
    return None
  if tok in stopwords:
    return None
  return stemmer.stem(tok)

def indexing(src, idx="ind"):
  if src[-1] != '/':
    src += '/'
  schema = Schema(docid=STORED(), content=TEXT(stored=True, analyzer=StandardAnalyzer()))
  ix = create_in(idx, schema)
  writer = ix.writer()

  files = os.listdir(src)
  for f in files:
    r = open(src + f, encoding="cp1252")
    terms = []
    for s in r:
      for sent in sent_tokenize(s.strip()):
        for tok in word_tokenize(sent):
          tok = preprocess(tok)
          if tok != None:
            terms.append(tok)
    r.close()
    cont = " ".join(terms)
    writer.add_document(docid="{}".format(f.split(".")[0]), content=cont)
  writer.commit()

In [6]:
indexing("Cranfield/Cranfield", "ind")


***2) Xử lý truy vấn***
- Chuẩn bị tập Ground Truth
- Chuẩn bị tập câu truy vấn.
- Truy vấn chỉ mục với mô hình xác suất, trọng số term tính theo TF_IDF

In [7]:
def readGroundTruth(src):
  if src[-1] != '/':
    src += '/'

  GT = {}
  for f in os.listdir(src):
    r = open(src + f)
    rel = {}
    for s in r:
      s = s.strip()
      sp = s.split("\t")
      if len(sp) < 2:
        continue
      did = sp[0].split(" ")[1]
      rel[did] = int(sp[1])
    GT[f.split(".")[0]] = rel
    r.close()
  return GT

In [8]:
GroundTruth = readGroundTruth("Cranfield/RES")
print(GroundTruth)

{'217': {'666': 1, '667': 1, '1258': 2, '1394': 2, '668': 3, '670': 3, '1204': 3, '1391': 4, '1395': 1, '1300': 3, '37': 3, '559': 3, '630': 3, '1107': 3, '1213': 3, '1191': -1}, '191': {'914': 1, '915': 1, '285': 2, '857': 2, '858': 2, '15': 3, '391': 3, '856': 3, '390': 4, '1008': 4, '948': 3, '864': 3, '658': 3, '894': -1}, '136': {'1021': 1, '1022': 4, '1034': 4, '951': -1}, '132': {'1014': 1, '1013': 3, '1015': 3, '1016': 3, '1012': 4, '1018': 4, '1019': 4, '1017': 3, '951': 3, '1023': 3, '1020': 4, '952': 4, '1024': 4, '1025': 3, '1026': 3, '950': -1}, '164': {'311': 2, '415': 3, '416': 3, '798': 3, '316': 4, '335': 4, '1364': 3, '1367': 3, '503': -1}, '67': {'2': 1, '3': 1, '664': 2, '629': 2, '323': 3, '324': 3, '393': 3, '128': 1, '180': 1, '394': 3, '659': 3, '389': 3, '4': 3, '1302': 3, '388': -1}, '11': {'27': 3, '28': 3, '262': 3, '160': 4, '20': 4, '263': 3, '654': 4, '495': -1}, '149': {'1057': 3, '1058': 3, '1059': 3, '1052': 4, '930': 3, '931': 3, '934': 3, '936': 3, '

In [9]:
def readQuery(src):
  qry = {}
  r = open(src)
  for s in r:
    s = s.strip()
    ps = s.split("\t")
    terms = []
    for sent in sent_tokenize(ps[1]):
      for tok in word_tokenize(sent):
        tok = preprocess(tok)
        if tok != None:
          terms.append(tok)
    qry[ps[0]] = " ".join(terms)
  return qry

In [10]:
Queries = readQuery("Cranfield/query.txt")
print(Queries)

{'1': 'similar law must obey construct aeroelast model heat high speed aircraft', '2': 'structur aeroelast problem associ flight high speed aircraft', '3': 'problem heat conduct composit slab solv far', '4': 'criterion develop show empir valid flow solut chemic react ga mixtur base simplifi assumpt instantan local chemic equilibrium', '5': 'chemic kinet system applic hyperson aerodynam problem', '6': 'theoret experiment guid turbul couett flow behaviour', '7': 'possibl relat avail pressur distribut ogiv forebodi zero angl attack lower surfac pressur equival ogiv forebodi angl attack', '8': 'method -dash exact approxim -dash present avail predict bodi pressur angl attack', '9': 'paper intern /slip flow/ heat transfer studi', '10': 'real-ga transport properti air avail wide rang enthalpi densiti', '11': 'possibl find analyt similar solut strong blast wave problem newtonian approxim', '12': 'aerodynam perform channel flow ground effect machin calcul', '13': 'basic mechan transon aileron b

In [11]:
def processQueries(ind, qry):

  idx = index.open_dir(ind)
  searcher = idx.searcher(weighting=scoring.BM25F())
  parser = qparser.QueryParser("content", idx.schema, group=qparser.OrGroup)

  RET = {}
  for key in qry:
    query = parser.parse(qry[key])
    results = searcher.search(query, limit=None)
    rel = {}
    for i in range(len(results)):
      rel[results[i]["docid"]] = results[i].score
    RET[key] = rel
  return RET

In [12]:
RunResults = processQueries("ind", Queries)

***3) Tính chỉ số***
- Sử dụng đối tượng thuộc lớp RelevanceEvaluator từ package pytrec_eval.
- Tính chỉ số MAP ("intAP") và MAPint ("11pt_avg")

In [13]:
print(pytrec_eval.supported_measures)
eval = pytrec_eval.RelevanceEvaluator(GroundTruth, ["infAP", "11pt_avg"])
Results = eval.evaluate(RunResults)

MAP = 0
MAP11PT = 0

for key in Results:
  value = Results[key]["infAP"]
  if not math.isnan(value):
    MAP += value
  value = Results[key]["11pt_avg"]
  if not math.isnan(value):
    MAP11PT += value

MAP /= len(Results)
MAP11PT /= len(Results)

print(MAP, MAP11PT)

{'Rprec', 'map_cut', 'set_relative_P', 'gm_bpref', 'Rndcg', 'set_F', 'num_rel', 'num_q', 'ndcg', '11pt_avg', 'set_map', 'recip_rank', 'bpref', 'ndcg_rel', 'set_recall', 'recall', 'success', 'map', 'relstring', 'num_ret', 'utility', 'binG', 'P', 'gm_map', 'num_rel_ret', 'infAP', 'relative_P', 'ndcg_cut', 'set_P', 'num_nonrel_judged_ret', 'iprec_at_recall', 'G', 'Rprec_mult', 'runid'}
0.35099879106595755 0.3218448003963703


**II - STANZA (STANFORD CORENLP)**

STANZA là phiên bản wrap của stanford corenlp trên Python. Stanza có thể được dùng để nhận dạng ranh giới câu, tách từ, gán nhãn từ loại, phân tích cú pháp cấu trúc ngữ đoạn và phân tích cú pháp phụ thuộc. Kết quả xử lý được đặt trong một đối tượng để từ đó truy xuất đến từng kết quả như mong muốn.

***1) Cài đặt stanza***

In [ ]:
%%capture
!pip install stanza

***2) Khởi tạo đối tượng Pipeline***

Đối tượng Pipeline có nhiệm vụ thực hiện các tác vụ đã được nêu ở trên. Pipeline có thể cài đặt cho một số ngôn ngữ, chẳng hạn tiếng Anh (chưa có tiếng Việt).

Lưu ý, đối tượng Pipeline chỉ nên khởi tạo một lần, đối tượng này sẽ được sử dụng để xử lý khi cần.

In [ ]:
%%capture
import stanza
stanza.download('en')#, proxies=proxies)
nlp = stanza.Pipeline('en')

***3) Xử lý với đối tượng Pipeline***

Truyền một đoạn hoặc một văn bản cho đối tượng Pipeline. Kết quả là một tài liệu đã được xử lý gồm các câu.

In [ ]:
doc = nlp("I have a book.  It is an exellent book. President Abraham Lincoln had writtten fiction StarTrek in Houston, Texas.")

Dùng thuộc tính constituency để lấy kết quả phân tích cú pháp cấu trúc ngữ đoạn

In [ ]:
doc.sentences[0].constituency

(ROOT (S (NP (PRP I)) (VP (VBP have) (NP (DT a) (NN book))) (. .)))

Dùng thuộc tính dependencies để lấy kết quả phân tích cú pháp phụ thuộc

In [ ]:
doc.sentences[2].dependencies

[({
    "id": 5,
    "text": "writtten",
    "lemma": "writt",
    "upos": "VERB",
    "xpos": "VBN",
    "feats": "Tense=Past|VerbForm=Part",
    "head": 0,
    "deprel": "root",
    "start_char": 70,
    "end_char": 78
  },
  'nsubj',
  {
    "id": 1,
    "text": "President",
    "lemma": "President",
    "upos": "PROPN",
    "xpos": "NNP",
    "feats": "Number=Sing",
    "head": 5,
    "deprel": "nsubj",
    "start_char": 40,
    "end_char": 49
  }),
 ({
    "id": 1,
    "text": "President",
    "lemma": "President",
    "upos": "PROPN",
    "xpos": "NNP",
    "feats": "Number=Sing",
    "head": 5,
    "deprel": "nsubj",
    "start_char": 40,
    "end_char": 49
  },
  'flat',
  {
    "id": 2,
    "text": "Abraham",
    "lemma": "Abraham",
    "upos": "PROPN",
    "xpos": "NNP",
    "feats": "Number=Sing",
    "head": 1,
    "deprel": "flat",
    "start_char": 50,
    "end_char": 57
  }),
 ({
    "id": 1,
    "text": "President",
    "lemma": "President",
    "upos": "PROPN",
    "xp

Mỗi câu có nhiều từ, để truy xuất đến từ, từ gốc và nhãn từ loại thì dùng các thuộc tính text, lemma và xpos

In [ ]:
doc.sentences[2].entities


[{
   "text": "Abraham Lincoln",
   "type": "PERSON",
   "start_char": 50,
   "end_char": 65
 },
 {
   "text": "StarTrek",
   "type": "WORK_OF_ART",
   "start_char": 87,
   "end_char": 95
 },
 {
   "text": "Houston",
   "type": "GPE",
   "start_char": 99,
   "end_char": 106
 },
 {
   "text": "Texas",
   "type": "GPE",
   "start_char": 108,
   "end_char": 113
 }]

In [ ]:
for word in doc.sentences[0].words:
  print(word.text, word.lemma, word.xpos)


I I PRP
have have VBP
a a DT
book book NN
. . .


**III - PY-VNCORENLP (VNCORENLP)**

PY-VNCORENLP là phiên bản wrap của vncorenlp trên Python. Py-vncorenlp có thể được dùng để nhận dạng ranh giới câu, tách từ, gán nhãn từ loại và phân tích cú pháp phụ thuộc. Kết quả xử lý được đặt trong một đối tượng để từ đó truy xuất đến từng kết quả như mong muốn.

***1) Cài đặt py-vncorenlp***

In [ ]:
%%capture
!pip install py-vncorenlp

***2) Khởi tạo đối tượng VnCoreNLP***

Đối tượng VnCoreNLP có nhiệm vụ thực hiện các tác vụ đã được nêu ở trên.

Lưu ý, đối tượng VnCoreNLP chỉ được khởi tạo một lần. Nếu khởi tạo một lần nữa sẽ bị lỗi do đối tượng trước đã chiếm dụng tài nguyên của máy, đối tượng sau sẽ không được tạo.

In [ ]:
import py_vncorenlp
py_vncorenlp.download_model(save_dir='./')
model = py_vncorenlp.VnCoreNLP(annotators=["wseg", "pos", "ner", "parse"], save_dir='./')

***3) Xử lý với đối tượng VnCoreNLP***

Truyền một đoạn hoặc một văn bản cho đối tượng VnCoreNLP. Kết quả là một tài liệu đã được xử lý gồm các câu.

In [ ]:
sentences = model.annotate_text("Hoa Kỳ là thị trường xuất khẩu lớn nhất của Huyện Định Quán . Vitamin tham gia vào quá trình chuyển hoá năng lượng của cơ thể")

Kết quả xử lý kết quả phân tích cú pháp phụ thuộc của các câu.

In [ ]:
sentences[0]

[{'index': 1,
  'wordForm': 'Hoa_Kỳ',
  'posTag': 'Np',
  'nerLabel': 'B-LOC',
  'head': 2,
  'depLabel': 'sub'},
 {'index': 2,
  'wordForm': 'là',
  'posTag': 'V',
  'nerLabel': 'O',
  'head': 0,
  'depLabel': 'root'},
 {'index': 3,
  'wordForm': 'thị_trường',
  'posTag': 'N',
  'nerLabel': 'O',
  'head': 2,
  'depLabel': 'vmod'},
 {'index': 4,
  'wordForm': 'xuất_khẩu',
  'posTag': 'V',
  'nerLabel': 'O',
  'head': 3,
  'depLabel': 'nmod'},
 {'index': 5,
  'wordForm': 'lớn',
  'posTag': 'A',
  'nerLabel': 'O',
  'head': 3,
  'depLabel': 'nmod'},
 {'index': 6,
  'wordForm': 'nhất',
  'posTag': 'A',
  'nerLabel': 'O',
  'head': 5,
  'depLabel': 'amod'},
 {'index': 7,
  'wordForm': 'của',
  'posTag': 'E',
  'nerLabel': 'O',
  'head': 3,
  'depLabel': 'nmod'},
 {'index': 8,
  'wordForm': 'Huyện',
  'posTag': 'N',
  'nerLabel': 'B-LOC',
  'head': 7,
  'depLabel': 'pob'},
 {'index': 9,
  'wordForm': 'Định_Quán',
  'posTag': 'Np',
  'nerLabel': 'I-LOC',
  'head': 8,
  'depLabel': 'nmod'},
 

In [ ]:
sentences[0][0]["posTag"]

'Np'

Mỗi câu gồm nhiều từ. Với mỗi từ, Dùng key 'wordForm' và 'posTag' để xác định biểu diễn từ đã được tách và nhãn từ loại của nó

In [ ]:
sent = sentences[0]
for word in sent:
  print(word['wordForm'], word['posTag'])

Hoa_Kỳ Np
là V
thị_trường N
xuất_khẩu V
lớn A
nhất A
của E
Huyện N
Định_Quán Np
. CH
